<a href="https://colab.research.google.com/github/kmachina/cv_course/blob/master/leaf_classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

os.environ['KAGGLE_USERNAME'] = 'kimbochen'
os.environ['KAGGLE_KEY'] = 'fcc31da31010e0b78a38e936d060dd8f'
CONTEST_NAME = 'leaf-classification'

!pip install --upgrade --force-reinstall --no-deps --quiet kaggle
!kaggle competitions download -q -c {CONTEST_NAME}
!unzip {CONTEST_NAME}.zip && rm -f {CONTEST_NAME}.zip
!unzip -q '*.zip' && rm -f *.zip

In [0]:
from __future__ import print_function, division

import torch
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.datasets import ImageFolder

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt